# Aprendizagem de Máquina (AM 2018.1)
Centro de Informática, CIn / UFPE<br/>
Professor: Cleber Zanchettin<br/>
Equipe:
* Cleison Amorim
* Dinaldo Pessoa

# Introdução
<p>
Este trabalho aborda um problema de classificação binária, conforme descrito pelo enunciado da competição. Cada uma das amostras do conjunto de dados disponibilizado está relacionada a usuários do seguro automotivo Porto Seguro e indicam quando esses usuários acionaram ou não o seguro no decorrer do último ano. Dado isso, o propósito é então elaborar um modelo que seja capaz de prever quando novas amostras de usuários desse seguro informarão sinistro ou não no próximo ano. É esperado que a performance do modelo elaborado seja comparável com o resultado médio da leaderboard do Kaggle.
</p>
Este documento está estruturado da seguinte forma: 
* Introdução, traz uma breve explicação do problema tratado, do objetivo do trabalho e prepara o contexto de execução da implementação;
* Análise dos dados, faz uma análise exploratória dos dados;
* Engenharia de features, realiza a engenharia das features que vão compor o modelo;
* Experimento, realiza a seleção do algoritmo de aprendizagem de máquina e a otimização dos hiperparâmetros do algoritmo escolhido;
* Conclusão, considerações finais sobre os resultados obtidos.

## Preparação do experimento
<p>
Nesta seção, apenas são realizados algumas inicializações básicas para o experimento, como importação de dependências comuns e configuração de parâmetros.
</p>

In [ ]:
import os
import pprint as pp
import math as ma
import numpy as np
import pandas as pd
from plotnine import *
from plotnine.data import *
from sklearn import base
from sklearn import tree
from sklearn import naive_bayes as nb
from sklearn import svm as svm
from sklearn import neighbors as nei
from sklearn import preprocessing as pre
from sklearn import feature_extraction as fe
from sklearn import pipeline as pi
from sklearn import model_selection as ms
from sklearn import ensemble as ens
from sklearn.metrics import make_scorer
import xgboost as xgb
import bayes_opt as bo
import category_encoders as ce

In [ ]:
# Nível de verbosidade do log:
verbose = 10

# Semente de estado randômico:
seed = round(ma.pi * 10**4)

In [ ]:
# Extrai tipos das colunas:
def cols_types_to_df(df, cols):
    return pd.DataFrame(df[cols].dtypes.values.reshape(1, -1), columns=cols, index=['dtype'])

# Análise dos dados
<p>
A questão apresenta um conjunto de dados de usuários de seguros automotivos da Porto Seguro. Para que fosse possível preservar o sigilo das informações dos usuários, todas as features tiveram seus nomes codificados e não foram fornecidos quaisquer detalhes adicionais quanto à semântica ou relação entre elas. Uma pré-visualização de seus primeiros registros permite-nos constatar isso:
</p>

In [ ]:
ID_COL = 'id'
TARGET_COL = 'target'
SPECIAL_COL_NAMES = [ID_COL, TARGET_COL]

# Carregando dados:
train = pd.read_csv('../input/train.csv').drop(ID_COL, axis=1)#.sample(frac=0.25, random_state=seed)
headers = train.columns

# Separando o rótulo:
X = train.drop(TARGET_COL, axis=1)
y = train[TARGET_COL]

train.head(10)

<p>
Para tentar suprir a ausência de informação semância para as features, foram conduzidas algumas pesquisas para tentar identificar variáveis importantes para as seguradoras, verificar se há um grau de relevância ou de interdependência entre elas, e para relacioná-las com o conjunto de dados da questão. Algumas variáveis extraídas a partir dessa pesquisa foram:
<ol>
    <li>Região geográfica;</li>
    <li>Idade;</li>
    <li>Gênero;</li>
    <li>Estado civil;</li>
    <li>Tempo de habilitação;</li>
    <li>Histórico de acidentes ao volante;</li>
    <li>Registro de sinistros;</li>
    <li>Histórico de crédito;</li>
    <li>Tipo do veículo;</li>
    <li>Uso do veículo;</li>
    <li>Quilometragem dirigidas anualmente;</li>
    <li>Cobertura de seguro anterior;</li>
    <li>Coberturas do seguro atual.</li>
</ol>
</p>
<p>
Infelizmente, mesmo após essa pesquisa e entendimento das variáveis acima não foi possível encontrar fatores que estabelecessem uma relação sólida entre essas variáveis e aquelas representadas pelas features. Por esse motivo, a tentantiva de encontrar a semântica para os dados foi descartada.
</p>

## Origem das features
<p>
Apesar da ausência de detalhes quanto à semântica, é possível observar que os dados apresentados pela Porto Seguro possuem em seu rótulo prefixos que os segregam em quatro origens distintas, conforme segue. Além disso, há algumas features calculadas, que aparentam ser informações processadas a partir de diversas outras variáveis relevantes para a seguradora nesse contexto.
<ul>
    <li><i>ps_ind</i>: features relacionadas ao condutor;</li>
    <li><i>ps_reg</i>: features relacionadas à região de residência;</li>
    <li><i>ps_car</i>: features relacionadas ao veículo propriamente;</li>
    <li><i>ps_calc</i>: features calculadas, criadas a partir de outras variáveis.</li>
</ul>
</p>

### Features do condutor

In [ ]:
IND_COL_NAMES = [x for x in headers if x.startswith('ps_ind')]
cols_types_to_df(train, IND_COL_NAMES)

### Features da região

In [ ]:
REG_COL_NAMES = [x for x in headers if x.startswith('ps_reg')]
cols_types_to_df(train, REG_COL_NAMES)

### Features do veículo

In [ ]:
CAR_COL_NAMES = [x for x in headers if x.startswith('ps_car')]
cols_types_to_df(train, CAR_COL_NAMES)

### Features calculadas

In [ ]:
CALC_COL_NAMES = [x for x in headers if x.startswith('ps_calc')]
cols_types_to_df(train, CALC_COL_NAMES)

## Tipos das features
<p>
As features também apresentam em seu rótulo sufixos que as discriminam quanto a três diferentes tipos:
<ul>
    <li><i>cat</i>: features categóricas;</li>
    <li><i>bin</i>: features binárias (no contexto da questão são features categóricas, mas que não possuem valores faltantes);</li>
    <li><i>(sem sufixo)</i>: demais features com valores contínuos ou ordinais.</li>
</ul>
</p>

### Features categóricas

In [ ]:
CAT_COL_NAMES = [x for x in headers if x.endswith('cat')]
cols_types_to_df(train, CAT_COL_NAMES)

### Features binárias

In [ ]:
BIN_COL_NAMES = [x for x in headers if x.endswith('bin')]
cols_types_to_df(train, BIN_COL_NAMES)

### Features contínuas

In [ ]:
LIN_COL_NAMES = [x for x in headers if (x not in (CAT_COL_NAMES + BIN_COL_NAMES + SPECIAL_COL_NAMES))]
cols_types_to_df(train, LIN_COL_NAMES)

<p>
Por fim, alguns detalhes adicionais apresentados pela questão serão relevantes para o experimento apresentado adiante:
<ul>
    <li><i>valores ausentes</i>: são representados no conjunto de dados pelo dígito -1;</li>
    <li><i>target</i>: coluna que contém as classes das amostras. O valor 1 indica uma amostra onde o usuário acionou o seguro no último ano, e o valor 0 indica que o seguro não foi acionado.</li>
</ul>
</p>

## Análise estatística
<p>
Uma análise de características de média, variância, mínimo, máximo e percentis permite-nos obter uma visão incial acerca do comportamento e distribuição dos dados disponibilizados:
</p>

In [ ]:
train.describe()

<p>
Os gráficos representam a classe <i>"1"</i> (usuário acionou o seguro) em azul, e a classe <i>"0"</i> (usuário não acionou o seguro) em vermelho. Eles permitem-nos confirmar que há um desbalanceamento dos dados que indica uma tendência menor dos condutores informarem sinistro. Esse é um comportamento até certo ponto esperado e aceitável para o contexto de seguros automotivos, quando levado em consideração o tamanho muito maior da base de condutores com seguro ativo e a menor proporção daqueles que efetivamente incidem em sinistro no decorrer de um ano. O desbalanceamento das classes foi tratado por meio de um parâmetro do algoritmo escolhido, o parâmetro scale_pos_weight do XGBClassifier. Nossa configuração aumentou o peso do erro de classificação da classe minoritária.
</p>

In [ ]:
target_counts = train[TARGET_COL].value_counts()
target_ratio = target_counts[0]/target_counts[1]
print("Proporção entre classes 0 e 1: %s" % target_ratio)

In [ ]:
(ggplot(train.astype(dtype={'target': object}), aes(x=TARGET_COL, fill=TARGET_COL))
 + geom_bar()   
 + facet_wrap(TARGET_COL)
 )

# Engenharia de features
<p>
No decorrer do experimento, foram aplicadas algumas abordagens para tratamento e codificação das features. A primeira delas foi de criar um transformador para substituir os valores faltantes, que após análise das demais etapas de tranformação dos dados foi detectado que a grande maioria deles já possui tratamento para isso e que seria desnecessário criar um transformador a parte. 
</p>
<p>
A segunda delas foi de tentar agrupar valores categóricos que apareciam com frequencia inferior a 5% dentro de um novo valor que significaria "outros". Isso foi motivado após verificar que técincas de codificação como a OneHot poderia chegar a duplicar o número de features das amostras, tornando ainda mais lenta a execução do algoritmo. Conforme esperado, essa estratégia alcançou o objetivo de controlar o crescimento da dimensionalidade das amostras, mas não foi capaz de contribuir positivamente para os melhores resultados do experimento.
</p>
<p>
A terceira (e provavelmente mais clássica) das abordagens foi a de aplicar a codificação OneHot para codificar os valores das features categórias. A implementação utilizada considera a possibilidade de tratar valores faltantes, e foi capaz de contribuir com bons resultados neste trabalho. Como alternativa para lidar com o problema da dimensionalidade, também tentou-se aplicar a estratégia codificação binária, mas seus resultados não superaram os da codificação OneHot neste experimento.
</p>
<p>
Por esse motivo, apenas a codificação OneHot foi incluída como passo anterior à classificação no pipeline de execução mostrado mais adiante.
</p>

In [ ]:
def drop_cols_func(X):
    '''Remove irrelevant columns.
    
    Parameters
    ----------

    X : pandas.DataFrame
        DataFrame with the features.
        
    Returns
    -------

    p : pandas.DataFrame
        DataFrame without *calc* features.
    '''
    return X.drop(columns=CALC_COL_NAMES)

drop_cols = pre.FunctionTransformer(func=drop_cols_func, validate=False)
drop_cols.transform(X).head()

In [ ]:
one_hot = ce.OneHotEncoder(cols=CAT_COL_NAMES, drop_invariant=True, handle_unknown='impute')
one_hot.fit_transform(X).head()

# Experimento
## Métrica de avaliação
<br/>
A métrica utilizada nesse trabalho segue a definição da competição do Kaggle, que é o Coeficiente de Gini Normalizado. Vale salientar que essa métrica é equivalente a área sobre a curva ROC [POWERS, 2011].

In [ ]:
def gini(actual, pred):   
    assert( len(actual) == len(pred) )
    all = np.asarray(np.c_[ actual, pred, np.arange(len(actual)) ], dtype=np.float)    
    # ordena por coluna da classe positiva de pred e por 
    all = all[ np.lexsort((all[:,2], -1*all[:,1])) ]    
    totalLosses = all[:,0].sum()
    giniSum = all[:,0].cumsum().sum() / totalLosses
    
    giniSum -= (len(actual) + 1) / 2.
    return giniSum / len(actual)

def gini_normalized(actual, pred):    
    return gini(actual, pred[:,1]) / gini(actual, actual)

gini_normalized_scorer = make_scorer(score_func=gini_normalized, greater_is_better=True, needs_proba=True)

def cv_metric(estimator, X, y, fit_params=None):    
    pred = ms.cross_val_predict(estimator, X, y, cv=ms.StratifiedKFold(n_splits=3, random_state=seed), verbose=verbose, fit_params=fit_params, method='predict_proba')
    return gini_normalized(y, pred) 

def xgb_metric(pred, dtrain):
    print('xgb_gini_normalized')
    return 'gini', gini_normalized(dtrain.get_labels(), pred)

xgb_fit_params = {        
    #'clf__silent': 1,
    'clf__eval_metric': xgb_metric
}

## Construção do pipeline
<br/>
Esse trabalho utilizou o modelo de programação do Scikit-learn [PEDREGOSA, 2011], o qual organiza o modelo de aprendizagem de máquina na forma de um pipeline. Um pipeline consiste na sequência de transformações das features seguida pelo algoritmo de aprendizagem. Essa composição cria um abstração que esconde os detalhes de engenharia de features. O modelo na forma de pipeline simplificou o reuso da solução em outras seções desse trabalho.

In [ ]:
xgb_params = {
    'objective': 'binary:logistic'
}

def pipeline():
    steps = [('drop_cols', drop_cols),
             ('one_hot', one_hot),
             ('clf', xgb.XGBClassifier(random_state=seed))]
    return pi.Pipeline(steps)

pipe = pipeline()

## Separação do conjunto de dados
<br/>
67% do conjunto de dados foram separados para seleção do algoritmo e para otimização de parâmetros do algoritmos escolhido, enquanto que os 33% restantes foram utilizados para avaliação do modelo otimizado.

In [ ]:
X_train, X_test, y_train, y_test = ms.train_test_split(X, y, test_size=0.33, random_state=seed)

## Seleção do algoritmo
<br/>
A seleção do algoritmo de aprendizagem de máquina pode ser feita de forma combinada com a otimização dos hiperparâmetros, Combined Algorithm Selection and Hyperparameter
optimization (CASH), ou de forma isolada, selecionar o algoritmo e depois otimizar os hiperparâmetros. A primeira abordagem é mais promissora, pois é sabido que os algoritmos possuem performance sensível a variação nos hiperparâmetros [GOMES, 2012]. Essa abordagem tem sido utilizada no ramo de AutoML [FEURER, 2015]. Porém, devido a indisponibilidade de bibliotecas de otimização adequadas para a tarefa no ambiente do Kaggle, esse trabalhou optou por escolher o algoritmo por meio de uma busca em grid com os parâmetros minimamente configurados e fazer a otimização de parâmetros do agoritmo escolhido em um segundo momento. 
<br/><br/>
Técnicas de otimização adequadas para resolver um problema de CASH trabalham com variáveis discretas e interdependentes, como é o caso do Sequential Model-Based Optimization for
General Algorithm Configuration (SMAC). Por exemplo, o algoritmo é uma variável de decisão discreta e os hiperparâmetros desse algoritmo são variáveis de decisão dependentes da escolha do mesmo. Essas variáveis de decisão dependentes só precisam estar ativas na busca quando a variável principal está ativa. A biblioteca à disposição no Kaggle usa a técnica de Bayesian Global Optimization with Gaussian Processes (GP) [SNOEK, 2012], que trabalha apenas com variáveis contínuas e não aproveita o conhecimento sobre a dependência entre elas. Mesmo não sendo adequada para o problema de CASH, ela pode ser utilizada para a otimização de hiperparâmetros de forma isolada, como é o caso desse trabalho.
<br/><br/>
Os algoritmos avaliados nesse trabalho foram KNeighborsClassifier com features normalizadas e configurado para utilizar apenas um vizinho na predição; e SVC (classificador SVM), GaussianNB (classificador bayesiano gaussiano) e XGBClassifier (classificador XGBoost) com as configurações padrões. KNeighborsClassifier, GaussianNB e SVC são implementações encontradas na biblioteca Scikit-learn [PEDREGOSA, 2011]. XGBClassifier vem de uma biblioteca focada em XGBoost [CHEN, 2016]. KNeighborsClassifier e SVC foi removido da implementação final desse trabalho após apresentar um tempo de treinamento proibitivo para o conjunto de dados utilizado. O KNeighborsClassifier rodou em 2h e obteve uma performance com duas ordens de grandeza abaixo do GaussianNB. O SVC não chegou a terminar de rodar, mas sua inviabilidade foi confirmada na documentação da biblioteca, que afirma que sua implementação do SVC não é escalável para conjuntos de dados com mais de 10.000 exemplos.
<br/><br/>
O algoritmo com o melhor resultado foi o XGBClassifier. Por isso, ele passou para a etapa de otimização de parâmetros, que será apresentada a seguir.

In [ ]:
# AVISO: desativado para não extrapolar o tempo de processamento do Kaggle
'''
param_grid = {'clf': [nb.GaussianNB(), xgb.XGBClassifier(random_state=seed, **xgb_params)]}
grid_search = ms.GridSearchCV(pipe, param_grid=param_grid, cv=ms.StratifiedKFold(n_splits=3, random_state=seed), verbose=verbose, scoring=gini_normalized_scorer, return_train_score=False)
grid_search.fit(X_train, y_train)
pp.pprint(grid_search.best_score_)
pp.pprint(grid_search.best_estimator_)
'''

## Otimização de hiperparâmetros
<br/>
A otimização de hiperparâmetros desse trabalho é focada no algoritmo escolhido, o XGBClassifier. Mesmo sendo realizada com apenas um algoritmo, esse é um procedimento bem dispendioso, pois o custo computacional de calcular a função objetivo da otimização corresponde a uma validação cruzada do modelo no conjunto de treinamento. Além disso, a biblioteca de otimização precisa calcular a função objetivo diversas vezes antes de alcançar um máximo local ou global. Considerando o alto custo e a necessidade de repetir esse procedimento a cada mudança nas features, esse trabalhou optou por realizar a otimização de hiperparâmetros de forma incremental e iterativa. Incremental porque quando as features não são alteradas, os dados da função objetivo são persistidos e reusados entre uma execução e outra da otimização. Iterativa porque quando as features são alteradas, é possível utilizar as melhores configurações de hiperparâmetros da iteração anterior para inicializar a busca com as novas features, conforme a abordagem de meta-learning [GOMES, 2012]. Vale salientar que, no caso desse trabalho, o conjunto de dados é o mesmo e isso facilita ainda mais a aplicação da abordagem de meta-learning.
<br/><br/>
Nesse trabalho, esse procedimento foi executado em duas iterações. A primeira iteração rodou durante um total de 24h, em um computador pessoal, com um conjunto de features que tem apenas uma transformação em relação ao conjunto de dados original, dados categóricos codificados por meio de One Hot Encoding. A segunda iteração corresponde a iteração final, com as novas features, e ela é a que se encontra pronta para ser reproduzida no kernel do Kaggle.

In [ ]:
if os.path.isfile('cache.csv'):
    # AVISO: atualize best_configs abaixo do 'else' sempre que houver uma busca ampla por melhores parâmetros
    best_configs = pd.read_csv('cache.csv').sort_values(by='target', ascending=False)
    best_configs = best_configs.head(5).drop(columns='target').to_dict(orient='list')
    pp.pprint(best_configs)
else:
    # melhores parâmetros hard-coded serão usados em ambientes sem o arquivo 'cache.csv'
    best_configs = {'colsample_bylevel': [1.0, 1.0, 1.0, 1.0, 0.6210536972328008],
         'colsample_bytree': [0.1639149309112309, 0.4719120160563106, 0.2738881448393379, 0.9597000000000002, 0.4998605743150732],
         'gamma': [9.311242926501441, 9.106399343640415, 9.84350195229832, 1.3079, 8.495594735754976],
         'learning_rate': [0.1, 0.1, 0.1, 0.1, 0.3],
         'max_delta_step': [0.0, 0.0, 0.0, 0.0, 9.937144736135796],
         'max_depth': [3.280267137915809, 3.744769687598659, 3.079866163690399, 3.8448, 3.0243950675489177],
         'min_child_weight': [1.0, 1.0, 1.0, 1.0, 1.0],
         'n_estimators': [496.3697387347585, 496.70662978009966, 362.50656654507236, 499.9892, 350.3584899983292],
         'reg_alpha': [9.79619399515934, 0.7119072322272911, 0.05797462113553141, 8.128199999999998, 4.196426179286616],
         'reg_lambda': [1.0, 1.0, 1.0, 1.0, 9.729243825652572],
         'scale_pos_weight': [2.091440738145689, 1.7334510787353241, 1.327362597470453, 2.6812, 1.072600468578576],
         'subsample': [0.7142488578466437, 0.9885285182085388, 0.7352809336635373, 0.9199, 0.8611579899201934]}
    # AVISO: comentar a linha de código abaixo para fazer mais exploração
    #best_configs = pd.DataFrame.from_dict(best_configs).head(1).to_dict(orient='list')

In [ ]:
def to_xgb_params(**kwargs):
    params = {}
    params['clf__max_depth'] = int(kwargs['max_depth']) 
    params['clf__learning_rate'] = max(kwargs['learning_rate'], 0)
    params['clf__n_estimators'] = int(kwargs['n_estimators'])
    params['clf__min_child_weight'] = int(kwargs['min_child_weight'])
    params['clf__max_delta_step'] = int(kwargs['max_delta_step'])
    params['clf__subsample'] = max(min(kwargs['subsample'], 1), 0)
    params['clf__colsample_bytree'] = max(min(kwargs['colsample_bytree'], 1), 0)  
    params['clf__colsample_bylevel'] = max(min(kwargs['colsample_bylevel'], 1), 0)
    params['clf__gamma'] = max(kwargs['gamma'], 0)
    params['clf__reg_alpha'] = max(kwargs['reg_alpha'], 0)
    params['clf__scale_pos_weight'] = max(kwargs['scale_pos_weight'], 0.001)
    return params

def xgb_cv(**kwargs):
    xgb = base.clone(pipe)
    xgb.set_params(**to_xgb_params(**kwargs))
    return cv_metric(xgb, X_train, y_train, fit_params=xgb_fit_params)
    
bo_opt = bo.BayesianOptimization(xgb_cv, {'max_depth': (3, 15),
                                      'learning_rate': (0.1, 0.0001),
                                      'n_estimators': (100, 500),
                                      'min_child_weight': (1, 1),
                                      'max_delta_step': (0, 10),
                                      'subsample': (0.5, 1.0),
                                      'colsample_bytree': (0.1, 1.0),
                                      'colsample_bylevel': (0.1, 1.0),
                                      'gamma': (0.0, 10.0),
                                      'reg_alpha': (0.0, 10.0),
                                      'reg_lambda': (1.0, 10.0),
                                      'scale_pos_weight': (1.0, target_ratio),
                                     },
                                    random_state=seed)
                                    
# AVISO: desativado para não extrapolar o tempo de processamento do Kaggle
'''
# manter True para execução de curta duração no Kaggle
# manter False para ampliar a busca por melhores parâmetros
fast_search = True
if fast_search:
    # usa melhores parâmetros como semente para o otimizador
    bo_opt.explore(best_configs)
else:
    if os.path.isfile('cache.csv'):
        # carrega dados da função objetivo
        bo_opt.initialize_df(pd.read_csv('cache.csv'))
    else:
        # explora configuração padrão e 5 configurações aleatórias        
        bo_opt.explore({'max_depth': [3],
                        'learning_rate': [0.1],
                        'n_estimators': [100],
                        'min_child_weight': [1],
                        'max_delta_step': [0],
                        'subsample': [1.0],
                        'colsample_bytree': [1.0],
                        'colsample_bylevel': [1.0],
                        'gamma': [0.0],
                        'reg_alpha': [0.0],
                        'reg_lambda': [1.0],
                        'scale_pos_weight': [1.0]})        
        bo_opt.maximize(init_points=5, n_iter=0, acq='ei')
        # explora as melhores parâmetros até então encontrados
        bo_opt.explore(best_configs)
        bo_opt.points_to_csv('cache.csv')        
# otimiza com 5 rodadas
bo_opt.maximize(init_points=0, n_iter=5, acq='ei')    
bo_opt.points_to_csv('cache.csv')
pp.pprint(bo_opt.res['max'])

# cria modelo otimizado
best_config = bo_opt.res['max']['max_params']
'''
# AVISO: comentar linha de código abaixo para usar resultado da otimização
best_config = {'colsample_bylevel': 0.45879276636748534,
                'colsample_bytree': 0.7824778832374746,
                'gamma': 5.489188804299045,
                'learning_rate': 0.1,
                'max_delta_step': 1.5254539296007386,
                'max_depth': 3.4967941454908598,
                'min_child_weight': 1.0,
                'n_estimators': 461.216046669665,
                'reg_alpha': 8.531590560075347,
                'reg_lambda': 9.89766553793669,
                'scale_pos_weight': 1.2379042177844632,
                'subsample': 0.798095844721928}
opt_params = to_xgb_params(**best_config)
opt_pipe = base.clone(pipe)
opt_pipe.set_params(**opt_params)

## Avaliação
<br/>
Mesmo com restrições de tempo de processamento, a otimização de hiperparâmetros realizada nesse trabalho conseguiu elevar a métrica de avaliação de 0.27510 (configuração padrão) para 0.28511 (configuração otimizada), o que representa um aumento de 3,6%. Para se ter um noção melhor, o aumento de performance entre a nossa configuração padrão e o primeiro colocado do leaderboard do Kaggle é de 8%. Esse resultado reforça a importância de uma boa configuração para os algoritmos de aprendizagem. Por outro lado, para aplicações práticas com uma escala maior do que a encontrada nesse trabalho, é preciso investir em paralelismo ou dispor de mais tempo de processamento para a biblioteca de otimização conseguir mapear uma função objetivo com tantos parâmetros de forma adequada, como é o caso da otimização dos hiperparâmetros do XGBClassifier.
<br/><br/>
Na primeira iteração desse trabalho, a biblioteca foi configurada para realizar exploration e exploitation, mas, na segunda iteração, devido às restrições de tempo de execução do ambiente do Kaggle, a biblioteca de otimização foi configurada para focar em exploitation. Uma extensão desse trabalho poderia configurar a biblioteca de otimização para realizar exploration em todas as iterações e optar por um meio termo no trade-off exploration-exploitation [BROCHU, 2010].

In [ ]:
pipe.fit(X_train, y_train, **xgb_fit_params)
print('Configuração padrão: {}'.format(gini_normalized(y_test, pipe.predict_proba(X_test))))
opt_pipe.fit(X_train, y_train, **xgb_fit_params)
print('Configuração otimizada: {}'.format(gini_normalized(y_test, opt_pipe.predict_proba(X_test))))

## Curvas de aprendizagem
<br/>
Com as curvas de aprendizagem, é possível avaliar de forma qualitativa se o modelo está sofrendo de overfitting ou underfitting. Além disso, o resultado do primeiro colocado no leaderboard do Kaggle, 0.29698, serve como referência para essa análise se ele for considerado como um topo de performance no conjunto de testes.
<br/><br/>
No gráfico abaixo, é possível observar que a performance no conjunto de treinamento está acima de 0.29698, o que indica que o modelo não sofre de underfitting. Em outras palavras, nosso modelo consegue uma performance no conjunto de treinamento melhor do que a performance do modelo do primeiro colocado no conjunto de testes do leaderboard do Kaggle. Após essa constatação, esse trabalho passou a focar na melhoria da performance no conjunto de testes, mesmo que isso representasse uma piora da performance no conjunto de treinamento. Por fim, a performance no conjunto de testes também não apresenta indícios de overfitting, mas, como é possível observar pelo leaderboard, há margem para melhorias.
![Curvas de aprendizagem](https://raw.githubusercontent.com/amorim-cleison/cin_am/develop/projeto_2/curvas_aprendizagem.png)

In [ ]:
# Wrapper implementado como workaround de um bug de "learning_cuve" do sklearn
# "learning_curve" não expõe o parâmetro "fit_params" e o Wrapper abaixo resolve isso
class FitParamsWrapper(base.BaseEstimator, base.ClassifierMixin):
    def __init__(self, estimator, fit_params):
        self.estimator = estimator
        self.fit_params = fit_params

    def fit(self, X, y=None):
        self.estimator.fit(X, y, **self.fit_params)
        return self

    def predict(self, X, y=None):
        return self.estimator.predict(X)

    def predict_proba(self, X, y=None):        
        return self.estimator.predict_proba(X)
    
    def score(self, X, y=None):        
        return self.estimator.score(X, y)

In [ ]:
# AVISO: desativado para não extrapolar o tempo de processamento do Kaggle
'''
sizes, train_scores, test_scores = ms.learning_curve(FitParamsWrapper(opt_pipe, xgb_fit_params), X, y, cv=ms.StratifiedKFold(n_splits=3, random_state=seed), scoring=gini_normalized_scorer, verbose=verbose, random_state=seed)
df_lc = pd.DataFrame(data={'m': sizes, 'train': train_scores.mean(axis=1), 'test': test_scores.mean(axis=1)})
df_lc = pd.melt(df_lc, id_vars=['m'], value_vars=['train', 'test'], var_name='type', value_name='score')
(ggplot(df_lc)
 + aes(x='m', y='score', fill='type', color='type')
 + geom_line()
 )
 '''

## Submissão
<br/>
Antes de realizar a predição e a submissão para o Kaggle, o modelo com hiperparâmetros otimizados passa pelo treinamento com o conjunto de dados completo. Isso é feito tendo em vista que quanto maior o conjunto de treinamento, maior as chances do modelo generalizar o conhecimento.

In [ ]:
opt_pipe.fit(X, y, **xgb_fit_params)
test = pd.read_csv('../input/test.csv')
X_test = test[X.columns]
targets = opt_pipe.predict_proba(X_test)
targets = pd.DataFrame({'id': test.id, 'target': targets[:,1]})
targets.to_csv('submission.csv', index=False)

# Conclusão
<br/>
O resulado final desse trabalho conseguiu alcançar os objetivos estabelecidos, que era prever quais usuários desse seguro informarão sinistro ou não no próximo ano com uma performance comparável com a média do leaderboard do Kaggle. No dia 11/07/2018, o score do nosso modelo otimizado ficou igual ao resultado da posição 3062, de um total de 5169 competidores.

# Referências bibliográficas
* CHEN, Tianqi; GUESTRIN, Carlos. Xgboost: A scalable tree boosting system. In: Proceedings of the 22nd acm sigkdd international conference on knowledge discovery and data mining. ACM, 2016. p. 785-794.
* FEURER, Matthias et al. Efficient and robust automated machine learning. In: Advances in Neural Information Processing Systems. 2015. p. 2962-2970.
* GOMES, Taciana AF et al. Combining meta-learning and search techniques to select parameters for support vector machines. Neurocomputing, v. 75, n. 1, p. 3-13, 2012.
* HUTTER, Frank; HOOS, Holger H.; LEYTON-BROWN, Kevin. Sequential model-based optimization for general algorithm configuration. In: International Conference on Learning and Intelligent Optimization. Springer, Berlin, Heidelberg, 2011. p. 507-523.
* PEDREGOSA, Fabian et al. Scikit-learn: Machine learning in Python. Journal of machine learning research, v. 12, n. Oct, p. 2825-2830, 2011.
* POWERS, David Martin. Evaluation: from precision, recall and F-measure to ROC, informedness, markedness and correlation. 2011.
* SNOEK, Jasper; LAROCHELLE, Hugo; ADAMS, Ryan P. Practical bayesian optimization of machine learning algorithms. In: Advances in neural information processing systems. 2012. p. 2951-2959.